# 0. Prepare Environment 

In [8]:
import dimcli
import numpy as np
import pandas as pd

In [9]:
dimcli.login(
    key="", endpoint="https://app.dimensions.ai"
)
dsl = dimcli.Dsl()

get_new_data = False

Searching config file credentials for 'https://app.dimensions.ai' endpoint..
ERROR: `dsl.ini` credentials file not found.
HowTo: https://digital-science.github.io/dimcli/getting-started.html#authentication


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [10]:
def get_split(ser: pd.Series):
    return np.array_split(ser, ser.shape[0] / 400)


def get_data():
    if get_new_data:
        control = pd.read_csv("control-pmid.csv")
        """Query:
        https://pubmed.ncbi.nlm.nih.gov/?term=%28%28%22machine+learning%22%5BMajr%5D+OR+%28%22machine%22%5Bti%5D+AND+%22learning%22%5Bti%5D%29+OR+%22machine+learning%22%5Bti%5D+OR+%22AI%22%5Bti%5D+OR+%22Artificial+Intelligence%22%5Bti%5D+OR+%22artificially+intelligent%22%5Bti%5D+OR+%22Artificial+Intelligence%22%5BMeSH%5D+OR+%22Algorithms%22%5BMeSH%5D+OR+%22algorithm*%22%5Bti%5D+OR+%22deep+learning%22%5Bti%5D+OR+%22computer+vision%22%5Bti%5D+OR+%22natural+language+processing%22%5Bti%5D+OR+%22neural+network*%22%5Bti%5D+OR+%22neural+networks%2C+computer%22%5BMeSH%5D+OR+%22intelligent+machine*%22%5Bti%5D%29+AND+%28exp+%22Intensive+Care+Units%22%2F+OR+exp+%22Critical+Care%22%2F+OR+%28ICU+OR+IC+OR+%28%28intensive+OR+critical%29+ADJ3+%28care+OR+therapy+OR+unit*+OR+patient*+OR+department%5C*%29%29%29%29%29&filter=years.2010-2023
        """
        df_pubs_control = pd.DataFrame()
        df_auths_control = pd.DataFrame()
        df_affils_control = pd.DataFrame()

        for pmid_list in get_split(control.PMID):
            query = fr"""search publications where pmid in {list(pmid_list)} and year in [2010:2022]
            return publications[basics+abstract+authors_count+doi+concepts_scores+times_cited+mesh_terms
            +journal+editors+field_citation_ratio+funder_countries+funders+open_access
            +relative_citation_ratio+publisher+pmid+supporting_grant_ids+research_org_cities+pmid
            +research_org_countries+research_org_country_names+research_org_names+research_org_state_codes
            +research_org_state_names+research_orgs+researchers+category_bra+category_for+category_hra+category_hrcs_hc+
            category_hrcs_rac+category_icrp_cso+category_icrp_ct+category_rcdc+category_sdg+category_uoa]"""
            data = dsl.query_iterative(query)
            df_pubs_control = pd.concat([df_pubs_control, data.as_dataframe()])
            df_auths_control = pd.concat(
                [df_auths_control, data.as_dataframe_authors()]
            )
            df_affils_control = pd.concat(
                [df_affils_control, data.as_dataframe_authors_affiliations()]
            )

        df_pubs_control.to_csv("control_group_pubs.csv")
        df_auths_control.to_csv("control_group_auths.csv")
        df_affils_control.to_csv("control_group_affils.csv")

        compare_query = fr"""search publications where year in [2010:2022] for "\"datamed org display item php\" OR \"physionet mimic ii database\" 
        OR \"mimic iii medical information mart for intensive care\" OR
        \"10.13026 s6n6 xd98\" OR \"mimic iv\" OR \"physionet org content mimiciv\" OR
        \"eicu crd\" OR \"mimic cxr\"" return publications[basics+abstract+authors_count+doi+concepts_scores+times_cited
            +mesh_terms+journal+editors+field_citation_ratio+funder_countries+funders+open_access
            +relative_citation_ratio+publisher+pmid+supporting_grant_ids+research_org_cities+pmid
            +research_org_countries+research_org_country_names+research_org_names+research_org_state_codes
            +research_org_state_names+research_orgs+researchers+category_bra+category_for+category_hra+category_hrcs_hc+
            category_hrcs_rac+category_icrp_cso+category_icrp_ct+category_rcdc+category_sdg+category_uoa]"""
        compare = dsl.query_iterative(compare_query)

        df_pubs_compare = compare.as_dataframe()
        df_auths_compare = compare.as_dataframe_authors()
        df_affils_compare = compare.as_dataframe_authors_affiliations()

        df_pubs_compare.to_csv("mimic_pubs.csv")
        df_auths_compare.to_csv("mimic_auths.csv")
        df_affils_compare.to_csv("mimic_affils.csv")

    else:
        df_pubs_compare = pd.read_csv("MIMIC Files/mimic_pubs.csv")
        df_auths_compare = pd.read_csv("MIMIC Files/mimic_auths.csv")
        df_affils_compare = pd.read_csv("MIMIC Files/mimic_affils.csv")

        df_pubs_control = pd.read_csv("MIMIC Files/control_group_pubs.csv")
        df_auths_control = pd.read_csv("MIMIC Files/control_group_auths.csv")
        df_affils_control = pd.read_csv("MIMIC Files/control_group_affils.csv")

    return (
        df_pubs_compare,
        df_auths_compare,
        df_affils_compare,
        df_pubs_control,
        df_auths_control,
        df_affils_control,
    )